In [62]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('../../code/')
import leakage
from pathlib import Path
import matplotlib
import seaborn as sns
import scipy.stats as st
from scipy.stats import mannwhitneyu

In [63]:
def inf_to_100000(model):
    for r in model.reactions:
        if np.isinf(r.lower_bound):
            r.lower_bound = -100000
        if np.isinf(r.upper_bound):
            r.upper_bound = 100000
    return model

def print_summary(model, solution):
    # Print uptake
    for r in model.exchanges:
        flux = solution[r.id]
        if  flux >1e-3:
            print(r.id, r.name,flux, r.bounds)
        elif flux < -1e-3:
            print(r.id, r.name,flux, r.bounds)
    #print("## Demands ##")
    #for r in model.demands:
    #    print(r.id, solution[r.id])

    print("Growth: ", solution['r_2111'])
    print("Prot pool exchange: ", solution['prot_pool_exchange'])

# Create model without reversible exchanges

In [40]:
model = cobra.io.read_sbml_model('../../models/yeast/ecYeastGEM_batch_8_3_4.xml')
model.solver = 'gurobi'
model = inf_to_100000(model)
for r in model.boundary:
    if r.id[-3:] == 'REV':
        # print(r, r.upper_bound)
        r_forward = model.reactions.get_by_id(r.id[:-4])
        if r.upper_bound != 0:
            r_forward.lower_bound = -r.upper_bound
            # print(r_forward,r_forward.name, r_forward.bounds)
        r.remove_from_model()
# fn = '../../models/yeast/ecYeastGEM_batch_8_3_4_no_rev_ex.xml'
# cobra.io.write_sbml_model(model, fn)

In [67]:
model = cobra.io.read_sbml_model('../../models/yeast/ecYeastGEM_batch_8_3_4_no_rev_ex.xml')
model.reactions.get_by_id('r_1714').lower_bound = -10.7


In [68]:
solution = model.optimize()
print_summary(model, solution)
m = model.metabolites.get_by_id('s_0362[c]')
print(m.name, m.summary())
s = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = (r_id, 3.15))
print(s)

r_1634 acetate exchange 3.150847674325645 (0.0, 100000.0)
r_1654 ammonium exchange -2.4924355891824574 (-100000.0, 100000.0)
r_1672 carbon dioxide exchange 16.851758040581636 (0.0, 100000.0)
r_1714 D-glucose exchange -10.7 (-10.7, 0.0)
r_1761 ethanol exchange 11.799081171861205 (0.0, 100000.0)
r_1793 formate exchange 0.015592320692474442 (0.0, 100000.0)
r_1832 H+ exchange -5.75706774966927 (-100000.0, 100000.0)
r_1992 oxygen exchange -4.775969344852018 (-100000.0, 0.0)
r_2005 phosphate exchange -0.0994867201243665 (-100000.0, 100000.0)
r_2020 potassium exchange -0.0013206214578045737 (-100000.0, 100000.0)
r_2033 pyruvate exchange 0.8399229677948377 (0.0, 100000.0)
r_2049 sodium exchange -0.0014443160296099605 (-100000.0, 100000.0)
r_2060 sulphate exchange -0.03330429050876709 (-100000.0, 100000.0)
r_2100 water exchange 12.452362066355578 (-100000.0, 100000.0)
Growth:  0.36380756413349136
Prot pool exchange:  0.103720024310123
acetate [cytoplasm] s_0362[c]
Formula: C2H3O2

Producing Rea

# Load model

In [64]:
model = cobra.io.read_sbml_model('../../models/yeast/ecYeastGEM_batch_8_3_4.xml')
model.solver = 'gurobi'
model = inf_to_100000(model)

In [65]:
model.reactions.get_by_id('r_1714_REV').upper_bound = 10.7
print_summary(model, model.optimize())

r_1634 acetate exchange 3.1508476743256453 (0.0, 100000)
r_1672 carbon dioxide exchange 16.85175804058164 (0.0, 100000)
r_1761 ethanol exchange 11.799081171861207 (0.0, 100000)
r_1793 formate exchange 0.015592320692474286 (0.0, 100000)
r_2033 pyruvate exchange 0.839922967794843 (0.0, 100000)
r_2100 water exchange 12.452362066355587 (0.0, 100000)
r_1654_REV ammonium exchange (reversible) 2.492435589182457 (0.0, 100000)
r_1714_REV D-glucose exchange (reversible) 10.7 (0.0, 10.7)
r_1832_REV H+ exchange (reversible) 5.757067749669276 (0.0, 100000)
r_1992_REV oxygen exchange (reversible) 4.775969344852017 (0.0, 100000)
r_2005_REV phosphate exchange (reversible) 0.09948672012437539 (0.0, 100000)
r_2020_REV potassium exchange (reversible) 0.0013206214578045737 (0.0, 100000)
r_2049_REV sodium exchange (reversible) 0.0014443160296099605 (0.0, 100000)
r_2060_REV sulphate exchange (reversible) 0.03330429050876709 (0.0, 100000)
Growth:  0.36380756413349136
Prot pool exchange:  0.103720024310123


In [ ]:
model.

In [20]:
model.reactions.get_by_id('r_1714_REV').upper_bound = 10.7
solution = model.optimize()
print_summary(model, solution)
solution.shadow_prices['s_0362[c]']
m = model.metabolites.get_by_id('s_0362[c]')
r_id = 'r_1634_REV'
s = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = (r_id, 3.15))

with model:
    model.reactions.get_by_id(r_id).bounds = (3.15, 3.15)
    print_summary(model, model.optimize())

r_1634 acetate exchange 3.1508476743256506 (0.0, 100000)
r_1672 carbon dioxide exchange 16.851758040581636 (0.0, 100000)
r_1761 ethanol exchange 11.799081171861198 (0.0, 100000)
r_1793 formate exchange 0.015592320692474591 (0.0, 100000)
r_2033 pyruvate exchange 0.8399229677948341 (0.0, 100000)
r_2100 water exchange 12.45236206635558 (0.0, 100000)
r_1654_REV ammonium exchange (reversible) 2.492435589182461 (0.0, 100000)
r_1714_REV D-glucose exchange (reversible) 10.7 (0.0, 10.7)
r_1832_REV H+ exchange (reversible) 5.757067749669302 (0.0, 100000)
r_1992_REV oxygen exchange (reversible) 4.775969344852023 (0.0, 100000)
r_2005_REV phosphate exchange (reversible) 0.0994867201243665 (0.0, 100000)
r_2020_REV potassium exchange (reversible) 0.0013206214578045743 (0.0, 100000)
r_2049_REV sodium exchange (reversible) 0.0014443160296099611 (0.0, 100000)
r_2060_REV sulphate exchange (reversible) 0.033304290508767105 (0.0, 100000)
Growth:  0.3638075641334915
Prot pool exchange:  0.103720024310123
r_

In [37]:
with model:
    for r in model.boundary:
        if r.id[-3:] == 'REV':
            # print(r, r.upper_bound)
            r_forward = model.reactions.get_by_id(r.id[:-4])
            if r.upper_bound != 0:
                r_forward.lower_bound = -r.upper_bound
                print(r_forward,r_forward.name, r_forward.bounds)
                r.remove_from_model()
    print("#####")
    solution.shadow_prices['s_0362[c]']
    m = model.metabolites.get_by_id('s_0362[c]')
    r_id = 'r_1634'
    s = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = (r_id, 3.15))
    print(s)
    print("#####")
    model.reactions.get_by_id(r_id).bounds = (10,10)
    print_summary(model, model.optimize())

r_1654: s_0420[e] <=>  ammonium exchange (-100000, 100000)
r_1714: s_0565[e] <--  D-glucose exchange (-10.7, 0.0)
r_1832: s_0796[e] <=>  H+ exchange (-100000, 100000)
r_1861: s_0925[e] <=>  iron(2+) exchange (-100000, 100000)
r_1992: s_1277[e] <--  oxygen exchange (-100000, 0.0)
r_2005: s_1324[e] <=>  phosphate exchange (-100000, 100000)
r_2020: s_1374[e] <=>  potassium exchange (-100000, 100000)
r_2049: s_1438[e] <=>  sodium exchange (-100000, 100000)
r_2060: s_1468[e] <=>  sulphate exchange (-100000, 100000)
r_2100: s_0805[e] <=>  water exchange (-100000, 100000)
r_4593: s_4200[e] <=>  chloride exchange (-100000, 100000)
r_4594: s_4201[e] <=>  Cu2(+) exchange (-100000, 100000)
r_4595: s_4202[e] <=>  Mn(2+) exchange (-100000, 100000)
r_4596: s_4203[e] <=>  Zn(2+) exchange (-100000, 100000)
r_4597: s_4204[e] <=>  Mg(2+) exchange (-100000, 100000)
r_4600: s_4199[e] <=>  Ca(2+) exchange (-100000, 100000)
#####
-0.00010884266087862748
#####
r_1634 acetate exchange 10.0 (10, 10)
r_1654 amm

In [26]:
# Data on glucose uptake https://ami-journals.onlinelibrary.wiley.com/doi/10.1111/1751-7915.13995
init_glucose_conc = 16 #g/L
glucose_mw = 180.156 #g/mol
conc_gluc_mM = (init_glucose_conc/glucose_mw)*1e3
km_glc = 0.28 #mmol l−1
vmax_glc = 10 #mmol gDW−1

r_glc = vmax_glc * conc_gluc_mM / (conc_gluc_mM + km_glc)
print(r_glc)

9.968571784677293


## Note
As also stated in https://ami-journals.onlinelibrary.wiley.com/doi/10.1111/1751-7915.13995 glucose uptake is not constrained by km (i.e. vmax = 10)


In [28]:
model.reactions.get_by_id('r_1714_REV').upper_bound = 10


In [29]:
solution = model.optimize()
print_summary(model, solution)

r_1634 acetate exchange 3.1279933187031577 (0.0, 100000)
r_1672 carbon dioxide exchange 14.497501304355927 (0.0, 100000)
r_1761 ethanol exchange 9.481473517387105 (0.0, 100000)
r_1793 formate exchange 0.015479223367907058 (0.0, 100000)
r_2033 pyruvate exchange 1.8210776215830649 (0.0, 100000)
r_2100 water exchange 13.349287065068731 (0.0, 100000)
r_1654_REV ammonium exchange (reversible) 2.4743569591727215 (0.0, 100000)
r_1714_REV D-glucose exchange (reversible) 10.0 (0.0, 10)
r_1832_REV H+ exchange (reversible) 4.105673859636615 (0.0, 100000)
r_1992_REV oxygen exchange (reversible) 5.234950808334606 (0.0, 100000)
r_2005_REV phosphate exchange (reversible) 0.09876510324015143 (0.0, 100000)
r_2020_REV potassium exchange (reversible) 0.0013110424633373764 (0.0, 100000)
r_2049_REV sodium exchange (reversible) 0.0014338398290494171 (0.0, 100000)
r_2060_REV sulphate exchange (reversible) 0.03306272119862749 (0.0, 100000)
Growth:  0.36116872268247285
Prot pool exchange:  0.103720024310123
